In [4]:
import rospy
import actionlib
import PyKDL
import tf2_ros
from tf_conversions import posemath

from move_base_msgs.msg import MoveBaseAction, MoveBaseGoal
from geometry_msgs.msg import Twist, Pose, Pose2D, PoseStamped
from nav_msgs.msg import Odometry, Path, OccupancyGrid
from visualization_msgs.msg import MarkerArray, Marker
from gazebo_msgs.msg import ModelState, ContactsState
from std_msgs.msg import String
from std_srvs.srv import Empty

from threading import Event
import copy
import trimesh
import numpy as np

from robo_gym_server_modules.robot_server.grpc_msgs.python import robot_server_pb2

from roomor.generator import CubeRoomGenerator